---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [152]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [153]:
for G in P1_Graphs:
    degree = G.degree()
    a = sorted(degree.items(), key=lambda x: x[1], reverse=True)
    b = [t[1] for t in a]
    print({i:b.count(i) for i in list(set(b))})
    print()

{2: 465, 3: 237, 4: 100, 5: 56, 6: 32, 7: 26, 8: 16, 9: 14, 10: 9, 11: 7, 12: 5, 13: 5, 14: 2, 15: 1, 16: 1, 17: 2, 18: 2, 19: 3, 20: 2, 21: 1, 22: 2, 26: 2, 27: 2, 29: 1, 30: 2, 38: 1, 39: 2, 49: 1, 100: 1}

{7: 2, 8: 23, 9: 166, 10: 615, 11: 171, 12: 22, 13: 1}

{2: 2, 3: 93, 4: 567, 5: 79, 6: 9}

{4: 259, 5: 133, 6: 89, 7: 58, 8: 47, 9: 25, 10: 19, 11: 13, 12: 15, 13: 14, 14: 8, 15: 7, 16: 5, 17: 9, 18: 4, 19: 1, 20: 1, 21: 4, 22: 6, 23: 1, 24: 2, 25: 1, 27: 6, 29: 3, 30: 1, 31: 2, 32: 2, 36: 1, 42: 2, 44: 2, 46: 3, 47: 2, 51: 1, 53: 1, 55: 1, 62: 1, 92: 1}

{2: 102, 3: 195, 4: 218, 5: 126, 6: 74, 7: 20, 8: 12, 9: 2, 11: 1}



In [154]:
cluster, path = [], []
for G in P1_Graphs:
    cluster.append(nx.average_clustering(G))
    path.append(nx.average_shortest_path_length(G))
       
print(cluster)
print(path)

[0.03167539146454044, 0.5642419635919628, 0.4018222222222227, 0.03780379975223251, 0.0033037037037037037]
[4.099161161161161, 5.089871871871872, 9.378702269692925, 3.1048046283934134, 5.0785509568313305]


In [155]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H'] # Your Answer Here

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    
    # Your Code Here
    df = pd.DataFrame(index=[i[0] for i in G.nodes(data=True)])
    df['Department'] = [str(i[1]['Department']) for i in G.nodes(data=True)]
    df['ManagementSalary'] = [i[1]['ManagementSalary'] for i in G.nodes(data=True)]
    df['degree'] = G.degree().values()
    df['clustering'] = nx.clustering(G).values()
    df['closeness'] = nx.closeness_centrality(G).values()
    df['betweenness'] = nx.betweenness_centrality(G).values()
    df['pagerank'] = nx.pagerank(G)
    
    X_train = df.dropna().drop(['ManagementSalary','ManagementSalary'], axis=1)
    X_test = df[np.isnan(df['ManagementSalary'])].drop(['ManagementSalary','ManagementSalary'], axis=1)
    y_train = df.dropna()['ManagementSalary']
    y_test = df[np.isnan(df['ManagementSalary'])]['ManagementSalary']
    
    from sklearn.linear_model import LogisticRegression
    
    clf = LogisticRegression().fit(X_train, y_train)
    y_proba = clf.predict_proba(X_test)

    return pd.Series([i[1] for i in y_proba], index=X_test.index) # Your Answer Here

In [6]:
salary_predictions()

1       0.170441
2       0.615325
5       0.962797
8       0.155351
14      0.375431
18      0.201356
27      0.302731
30      0.358585
31      0.217413
34      0.163208
37      0.146592
40      0.275441
45      0.204160
54      0.288554
55      0.231247
60      0.239573
62      0.994888
65      0.456793
77      0.084679
79      0.139261
97      0.121075
101     0.095013
103     0.212007
108     0.139856
113     0.485623
122     0.074579
141     0.440978
142     0.913746
144     0.111829
145     0.449535
          ...   
913     0.056496
914     0.075696
915     0.012999
918     0.095832
923     0.047519
926     0.076626
931     0.061416
934     0.034308
939     0.022658
944     0.014970
945     0.044798
947     0.090361
950     0.121900
951     0.052074
953     0.038539
959     0.011377
962     0.012823
963     0.121554
968     0.080269
969     0.074347
974     0.056471
984     0.032351
987     0.083135
989     0.072705
991     0.097726
992     0.012005
994     0.014026
996     0.0129

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [41]:
def new_connections_predictions():
    
    # Your Code Here
    future_connections['preferential_attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['common_neighbor'] = list(map(lambda x: len(list(nx.common_neighbors(G,x[0],x[1]))), future_connections.index))
    future_connections['jaccard'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['resource_allocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    
    X_train = future_connections.dropna().drop('Future Connection', axis=1)
    y_train = future_connections.dropna()['Future Connection']
    X_test = future_connections[np.isnan(future_connections['Future Connection'])].drop('Future Connection', axis=1)
    
    from sklearn.linear_model import LogisticRegression

    clf = LogisticRegression().fit(X_train, y_train)
    y_proba = clf.predict_proba(X_test)
    
    return pd.Series([i[1] for i in y_proba], index=X_test.index) # Your Answer Here

In [42]:
new_connections_predictions()

(107, 348)    0.038475
(542, 751)    0.014602
(20, 426)     0.641016
(50, 989)     0.014881
(942, 986)    0.015184
(324, 857)    0.014842
(13, 710)     0.164249
(19, 271)     0.108506
(319, 878)    0.014978
(659, 707)    0.014631
(49, 843)     0.015066
(208, 893)    0.014697
(377, 469)    0.006408
(405, 999)    0.026881
(129, 740)    0.022631
(292, 618)    0.023324
(239, 689)    0.014997
(359, 373)    0.009125
(53, 523)     0.040020
(276, 984)    0.015051
(202, 997)    0.015110
(604, 619)    0.049905
(270, 911)    0.014993
(261, 481)    0.079897
(200, 450)    0.857455
(213, 634)    0.014517
(644, 735)    0.049063
(346, 553)    0.013842
(521, 738)    0.012608
(422, 953)    0.022178
                ...   
(672, 848)    0.014993
(28, 127)     0.972453
(202, 661)    0.013729
(54, 195)     0.999927
(295, 864)    0.014798
(814, 936)    0.014559
(839, 874)    0.015184
(139, 843)    0.014823
(461, 544)    0.011795
(68, 487)     0.011657
(622, 932)    0.014731
(504, 936)    0.019051
(479, 528) 